In [41]:
import mlflow
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
import dvc.api

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import RandomizedSearchCV

In [35]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(f"/Equipo_4/Absenteeism_at_work")

<Experiment: artifact_location='mlflow-artifacts:/9', creation_time=1728862130699, experiment_id='9', last_update_time=1728862130699, lifecycle_stage='active', name='/Equipo_4/Absenteeism_at_work', tags={}>

In [42]:
def load_data(filepath):
    print("1.- Loading and exploring the data")
    data = pd.read_csv(filepath,sep=";")
    return data

def prepared_data(data):
    print("2.- Prepared the data")
    print(data.describe())

    print("Número de filas en el dataset:", str(data.shape[0]))
    print("Número de columnas en el dataset:", str(data.shape[1]))

    # Mostrar las primeras filas del conjunto de datos
    print("Primeras filas del conjunto de datos:")
    print(data.head())

    # Paso 2: Limpieza de datos - verificar valores nulos
    print("\nRevisar valores nulos:")
    missing_data_summary = data.isnull().sum()
    print(missing_data_summary)

    # Eliminar filas con valores nulos (si las hubiera)
    df_cleaned = data.dropna()

    # Excluir 'ID' si fuera necesario
    # df_cleaned = df_cleaned.drop(columns=['ID'])   

    # Verificar si hay datos nulos después de la limpieza
    print("\nValores nulos después de limpieza:")
    print(df_cleaned.isnull().sum())

    # Paso 3: Verificar si hay columnas categóricas que deben transformarse
    categorical_columns = df_cleaned.select_dtypes(include=['object']).columns

    # Mostrar columnas categóricas identificadas
    if len(categorical_columns) > 0:
        print("\nColumnas categóricas que podrían necesitar transformación:")
        print(categorical_columns)
    else:
        print("\nNo hay columnas categóricas de tipo 'object' que necesiten transformación.")

    # Resumen final de la estructura del DataFrame después de limpieza
    print("\nResumen del DataFrame limpio:")
    print(df_cleaned.info())

    # Retornar el DataFrame limpio
    return df_cleaned

In [51]:
def scale_features(data, numeric_columns):
    # Verificar si el DataFrame data es None
    if data is None:
        raise ValueError("El DataFrame proporcionado es None. Verifica que los datos se hayan cargado correctamente.")
   
    # Verifica si las columnas numéricas existen en el DataFrame
    missing_cols = [col for col in numeric_columns if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Las siguientes columnas no se encuentran en el DataFrame: {missing_cols}")

    # Asegurarse de que las columnas seleccionadas sean numéricas
    if not all(np.issubdtype(data[col].dtype, np.number) for col in numeric_columns):
        raise ValueError("Una o más columnas no son numéricas. Revisa los tipos de datos en el DataFrame.")

    scaler = StandardScaler()
    data[numeric_columns] = scaler.fit_transform(data[numeric_columns])
    print("\nDatos después de la normalización:")
    print(data[numeric_columns].head())
    return data

def encode_categorical_columns(data, categorical_columns):
    label_encoder = LabelEncoder()
    for col in categorical_columns:
        data[col] = label_encoder.fit_transform(data[col])
    print("\nDatos después de la codificación de variables categóricas:")
    print(data[categorical_columns].head())
    return data

def apply_pca(data, numeric_columns, n_components=2):
    pca = PCA(n_components=n_components)
    pca_result = pca.fit_transform(data[numeric_columns])
    data['PCA1'] = pca_result[:, 0]
    data['PCA2'] = pca_result[:, 1]

    
    
    return data

# Función principal que ejecuta el preprocesamiento
def preprocess_data(df,numeric_columns,categorical_columns):
    df = scale_features(df, numeric_columns)
    
    df = encode_categorical_columns(df, categorical_columns)
    
    df = apply_pca(df, numeric_columns)

    print("\nResumen del DataFrame después del preprocesamiento:")
    print(df.head())
    
    return df

In [47]:
# Función para entrenar los modelos
def split_data(data, target, test_size=0.2, random_state=42):
    X = data.drop(target, axis=1)
    y = data[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

In [44]:
import mlflow
logged_model = 'runs:/48dab183113f498484e7262ff115880f/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [49]:
# Main function for running the pipeline
def main(filepath):
    #1.- Manipulación y preparación de datos
    data = load_data(filepath)
    df_cleaned  = prepared_data(data)
   
    numeric_columns = ['Transportation expense', 'Distance from Residence to Work', 'Service time', 'Age', 'Work load Average/day ', 'Hit target']
    categorical_columns = ['Month of absence', 'Day of the week', 'Seasons', 'Education', 'Disciplinary failure', 'Social drinker', 'Social smoker']

    df_preprocessed = preprocess_data(df_cleaned,numeric_columns,categorical_columns)
   
    
    #4.- Construcción, ajuste y evaluación de Modelos de Machine Learning
    print("\n### Entrenamiento y Evaluación Inicial ###")
    X_train, X_test, y_train, y_test = split_data(df_preprocessed, 'Absenteeism time in hours')

    loaded_model.predict(X_test)
    

In [52]:
main(filepath=r'D:\Smartware\Google Drive\Documentos\Tec de Monterrey\MNA\OAA\Semana 5\data\Absenteeism_at_work.csv')

1.- Loading and exploring the data
2.- Prepared the data
               ID  Reason for absence  Month of absence  Day of the week  \
count  740.000000          740.000000        740.000000       740.000000   
mean    18.017568           19.216216          6.324324         3.914865   
std     11.021247            8.433406          3.436287         1.421675   
min      1.000000            0.000000          0.000000         2.000000   
25%      9.000000           13.000000          3.000000         3.000000   
50%     18.000000           23.000000          6.000000         4.000000   
75%     28.000000           26.000000          9.000000         5.000000   
max     36.000000           28.000000         12.000000         6.000000   

          Seasons  Transportation expense  Distance from Residence to Work  \
count  740.000000              740.000000                       740.000000   
mean     2.544595              221.329730                        29.631081   
std      1.111831       